# Initialization

*Please read carefully*. In order to run the notebook successfully, you need to follow a few steps:
1. Download the raw data you'd like to load to the appropriate location on your computer.
2. Run the code below to import the necessary packages.
3. Determine whether you would like to load data to your local instance of the app (the one you run via `flask run` in your terminal at http://127.0.0.1:5000) or to the live online database (https://sspi.world).

## 1. Set the Filepath

In order to load data, you first have to tell your computer where it is.  I keep my data in the `bulk_data` folder, which is a sibling of this file (it is in the same folder).  You will notice, however, that you can't see any of my data files.  This is not an accident.  GitHub does not like to store large files, so I have `.gitignore`d the contents of the `bulk_data` folder. What this means is that git will not keep track of the contents of this folder, and will not upload or download it.  This means that it will be your responsibility to download the data files from the Google Drive and to organize the folder appropriately.  I recommend following the SSPI hierarchy, with three folders for the three pillars, nested folders for the categories, and further nested folders for the indicators.  Inside of the indicator folders, you can keep the raw(s) file for the data.

To import data from this folder, you'll use a relative import statement. It will start at the current directory (the one this file is in) and then move down the folder hierarchy.  The import statement below looks in the current folder with `./`, then looks to the subfolder `bulk_data`, then down to `SUS` for the Sustainability Pillar, then `LND` for the Land Category, and finally in the `NITROG` folder for the Sustainable Nitrogen Management indicator.  Into that folder I have downloaded the file from the source below, which itself contains a folder structure.  It have intentionally not cleaned up any of the files or folder structure from the download to make it as replicable as possible.

In [122]:
datapath = r"./raw_bulk_data/SUS/LND/NITROG/2022-epi-indicators-time-series-csv/2022-epi-indicators-time-series/SNM_ind.csv"

## 2. Import Packages

The code below imports the standard python data manipulation packages.  If you need to add a package for something, you can add it here.

In [123]:
import pandas as pd
import numpy as np
import re

## 3. Connect to the Database

This part is only important once you've run your analysis and would like to connect up to the database. The code below imports the `SSPIDatabaseConnector`, which is a Python class that allows us to post data directly to our databases. You can use this class to post data to both your development `sspi_bulk_data` (the one that lives on your machine that interfaces with the app you get when you run `flask run`) and the production `sspi_bulk_data` database, which lives on the server which hosts the live app at https://sspi.world. In order to access the production database, you'll need to ask me for your API token.  To access your development database, you can find the API token associated with your login credentials (the ones you use to login into the backend of the site) by visiting http://127.0.0.1:5000/auth/query while logged in, which will print out your API token.  

**It is vitally important that you _NEVER_ commit your API Tokens! They should only ever be stored in your local `.env` file, which is `.gitignore`d in this repository.** Never paste an API Token in any file unless you are absolutely sure it's the right place. Just don't do it!

In [124]:
from sys import path
path.append('../')

from database_connector.SSPIDatabaseConnector import SSPIDatabaseConnector
database = SSPIDatabaseConnector()

# I. Data Source

The 2018 SSPI originally used the Nitrogen Management Index sourced through the Environmental Performance Index (EPI) from their website, https://epi.yale.edu/.  The original values sourced for the SSPI were copied and pasted by hand from the EPI website.  The EPI website has since been updated, and the page from which the data were copied no longer exists.  The 2022 version of the page is available at https://epi.yale.edu/epi-results/2022/component/snm.

Fortunately, https://epi.yale.edu/downloads has a blurb which contains a link to https://sedac.ciesin.columbia.edu/data/set/epi-environmental-performance-index-2022/data-download for bulk data downloads.  The most recent bulk download is for 2022, and covers the years 1950 - 2022.

There are also previous years available for download.  It may be an interesting project to compare the EPI datasets over the years to see how they revise data and whether we think their methodology produces reliable results.  For now, we will focus on the 2022 dataset, which I have downloaded locally and uploaded to the Google Drive at https://drive.google.com/drive/folders/102Lg4NREiRhH_pzLabiVbkHvxRiaepDq?usp=sharing

In [125]:
SNM_raw_indicators = pd.read_csv(datapath)
SNM_raw_indicators

,code,iso,country,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,4,AFG,Afghanistan,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,8,ALB,Albania,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,12,DZA,Algeria,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,20,AND,Andorra,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,24,AGO,Angola,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,876,WLF,Wallis and Futuna Islands,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,732,ESH,Western Sahara,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,887,YEM,Yemen,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,894,ZMB,Zambia,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


# II. Cleaning the Data

This data is really friendly.  Everything is well labelled, none of the datatypes are looking strange (numbers look like numbers, strings look like strings), and there doesn't seem to be much missing data.  To clean it, we basically just need to rearrange it into a nice format. But what makes something more or less nice?

Right now, the data are in "wide format." There are a lot of columns, and the rows and columns carry information about the data. For instance, all of the year information is contained in the column titles. Were I to chop off the column titles, you would have no idea what year any of our observations correspond to anymore. Wide data formats are extremely common in excel. In fact, our static SSPI dataset is all in wide format, with different indicators in the different column titles. It's pretty friendly to work with wide data in excel, since it all fits on our rectangular screens, and it's really easy for humans to read off the data values when the data is wide. So what's wrong with it?

The unfortunate truth is that wide data is a little bit of a nightmare to program with for a couple of reasons.
* Wide data makes it finnicky to find your data values because the information isn't all in one place. To look up a data value, we first have to ask about the column titles, which contain some of the relevant information, keep track of that info, and then we have to go looking in the right column for the data. When humans look at a wide data table, we keep track of that information without much effort, but when we're programming, we must explicitly keep track of that extra information, which requires some work. 
* Wide data makes it harder to report a value as missing, since we have to explicitly write something in for that row column pair.  The dataset we're using here actually encodes missing values with large negative numbers like -8888 or -7777, which correspond to particular reasons why the data is missing. That can be a useful convention in certain circumstances, but when we're aggregating, sometimes its nice to not have to represent and keep track of missing data. In long format, that row simply isn't there.
* There are probably a bunch more reasons which I didn't think of when writing this, but I'd encourage you to try writing some code to work with wide data and notice how it becomes kind of hard to do things that feel like they should be easy.

For the SSPI, our main data cleaning task will be to pivot the table into long format. This will make it so that each row is a self-contained observation. Every observation can be uniquely identified by three pieces of information: a CountryCode, an IndicatorCode, and a Year.

## 1. Dropping Columns

Our first step will be to drop some pieces of information we won't need, which include the M49 Code and the Country Name (we'll match country codes to names later to ensure consistency of name spelling).

The `pandas` syntax here is a little non-intuitive to me. Start on the right side of the `=`: we call the `drop` method (a method is a function associated with an object) of the `SNM_raw_indicators` dataframe, which we pass a list of columns to drop. Thus, the expression right of the equals sign evaluates to a dataframe which is the same as `SNM_raw_indicators` but without the columns we dropped.  We then overwrite `SNM_raw_indicators` with that returned dataframe to move forward. We'll see an alternative syntax for this later.

**Note:** *In the code below, I write the database name again at the end of every line in order to print it out.  This is not strictly nessary to make the code work, it's just so that we can see what we're doing as we make changes.*

In [126]:
SNM_raw_indicators = SNM_raw_indicators.drop(columns=['code', 'country'])
SNM_raw_indicators

,iso,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,SNM.ind.2002,SNM.ind.2003,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,AFG,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,49.757632,49.370340,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,ALB,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,28.272164,26.632317,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,DZA,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,40.608845,32.869971,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,AND,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,AGO,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,33.645186,33.379226,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,WLF,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,7.916157,7.634675,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,ESH,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,18.044591,18.741216,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,YEM,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,41.175425,39.726230,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,ZMB,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,25.658239,34.430729,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


## 2. Renaming Columns

Next, we rename the `iso` column in the dataset to `CountryCode`, which is our standard way of referring to countries in the SSPI. The syntax for the `rename` method looks just like the syntax for `drop` above, except that the `columns` argument takes a dictionary instead of a list. That makes sense: how would we know which column to rename to what if we didn't have a dictionary? Dictionaries are great at specifying mappings, like the mapping between old column names and new column names.

In [127]:
SNM_raw_indicators = SNM_raw_indicators.rename(columns={'iso': 'CountryCode'})
SNM_raw_indicators

,CountryCode,SNM.ind.1995,SNM.ind.1996,SNM.ind.1997,SNM.ind.1998,SNM.ind.1999,SNM.ind.2000,SNM.ind.2001,SNM.ind.2002,SNM.ind.2003,...,SNM.ind.2013,SNM.ind.2014,SNM.ind.2015,SNM.ind.2016,SNM.ind.2017,SNM.ind.2018,SNM.ind.2019,SNM.ind.2020,SNM.ind.2021,SNM.ind.2022
0,AFG,44.015857,44.940128,46.931886,47.409220,45.026760,46.935768,46.024914,49.757632,49.370340,...,49.654873,52.474220,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194,50.972194
1,ALB,32.740191,32.877496,35.064258,32.049513,32.014128,33.975968,32.473496,28.272164,26.632317,...,40.553903,40.455314,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271,37.583271
2,DZA,39.122933,33.470399,28.022457,38.560891,37.182622,27.406628,40.748135,40.608845,32.869971,...,49.653891,42.949184,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487,44.832487
3,AND,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,AGO,24.672298,24.974387,25.923148,27.375270,28.690846,30.976644,32.359858,33.645186,33.379226,...,34.652635,30.103431,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713,29.331713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,WLF,6.511117,6.690118,6.809275,6.930870,7.049151,7.223545,7.367040,7.916157,7.634675,...,9.631419,9.441148,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377,9.452377
216,ESH,22.890672,21.792781,23.324610,21.604391,23.701670,18.721917,8.913881,18.044591,18.741216,...,15.001656,15.020989,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210,14.788210
217,YEM,40.789860,40.570173,39.305332,42.575937,42.763620,42.499581,42.102089,41.175425,39.726230,...,40.567349,39.558267,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760,34.527760
218,ZMB,26.058138,41.955960,31.324801,34.640796,41.127625,44.265747,37.610371,25.658239,34.430729,...,36.999181,40.853596,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438,36.637438


## 3. Pivoting the Data

Here comes the big one! In just a single line of code, we're going to transform the data from wide to long. It is totally possible to perform operations like this in Excel and Google Sheets, but I find it less intuitive and it can often be destructive to your data if you're not careful.  It also usually depends on the specification of options in dialogue boxes and other arcane knowledge of the software.  In `pandas`, we can do it in one line of code, and it's pretty easy to understand what's going on, so let's dig in.

All we need to do is call the `melt` method, which is a creative name. I've also heard this operation called "pivoting" the data and "reshaping" the data. How does it work? Well, we pass it a few arguments:
* `id_vars` is a list of columns which will act as our identifiers. In this case, we just have one (`CountryCode`), but you could imagine cases (e.g. Country and City) in which you might need multiple columns to uniquely identify an observation.
* `var_name` is the name of a new variable we will create which will contain the column names of the original dataframe.  In this case, we'll call it `YearString`, since it contains the year information in string format, along with some other junk we'll strip out later. We could've called it anything we wanted though.
* `value_name` is the name of a new variable we will create which will contain the data values from the original dataframe.  In this case, we'll call it `Raw`, since it contains the raw data values. Again, we could've called it anything we wanted.

When we call `melt`, we set aside our `id_vars`, and anything that isn't an id variable gets pulled out into the `var_name` and `value_name` columns.  The `var_name` column will contain the column names from the original dataframe, and the `value_name` column will contain the data values from the original dataframe. We now have long data!

In [128]:
SNM_long_indicators = SNM_raw_indicators.melt(id_vars=['CountryCode'], var_name='YearString', value_name='Raw')
SNM_long_indicators

,CountryCode,YearString,Raw
0,AFG,SNM.ind.1995,44.015857
1,ALB,SNM.ind.1995,32.740191
2,DZA,SNM.ind.1995,39.122933
3,AND,SNM.ind.1995,-8888.000000
4,AGO,SNM.ind.1995,24.672298
...,...,...,...
6155,WLF,SNM.ind.2022,9.452377
6156,ESH,SNM.ind.2022,14.788210
6157,YEM,SNM.ind.2022,34.527760
6158,ZMB,SNM.ind.2022,36.637438


## 4. Extracting the Year Information

What's so great about long data? Well, here's an example.

Suppose I asked you to pull out the year information from the column titles above when the data was in wide format. How would you go about doing this? 
* I showed you how to use the `rename` above, so you could just type out the renames by hand. This is not a good method.
* Maybe you could find a way to loop through the column names and rename each one. That would be a little tricky, but it's certainly possible.

Instead of having to hack together a solution in wide format, in long format we can just define a new column, and because each row contains all the information you need, we don't need to go looking in other places for it.

The code below uses a python list comprehension for compactness. List comprehensions are a nice way of building lists quickly. Here's what you need to know:
```python
[expression for item in iterable]
```
In our case, we have:
* `expression` is `re.search(r"\d{4}", s).group(0)`, which is a function of `s` which I'll talk about in a second.
* `item` is `s`, which is a string containing the column name.
* `iterable` is `SNM_raw_indicators["YearString"]`, which is a column of strings containing the former column names.

What we're doing is building a list `s` by `s` for each `s` in `SNM_raw_indicators["YearString"]`, and what we're doing with our `expression` is extracting the year information from each `s` with each. The `re.search(r"\d{4}", s).group(0)` is  regular expression which searches for a string of four digits in `s` and returns it.
* The `re` module is the regular expression (regex) module in python.  It contains a bunch of functions for working with regular expressions.
* The `r` before the string (the first argument of the search method of the `re` module) tells python that this is a "raw" string, which means that it will ignore any special characters in the string.  This is important because `\d` is a special character in python, and we want it to be interpreted as a literal `\d` instead of as a special character.  If we didn't put the `r` there, python would interpret `\d` as a special character and would throw an error.
* 
* The `group(0)` is necessary because `re.search` returns a `Match` object, which is a special type of object that contains information about the match.
* We want the actual string, so we call the `group` method of the `Match` object to get it.

In [129]:
SNM_long_indicators["Year"] = [re.search(r"\d{4}", s).group(0) for s in SNM_long_indicators["YearString"]]
SNM_long_indicators

,CountryCode,YearString,Raw,Year
0,AFG,SNM.ind.1995,44.015857,1995
1,ALB,SNM.ind.1995,32.740191,1995
2,DZA,SNM.ind.1995,39.122933,1995
3,AND,SNM.ind.1995,-8888.000000,1995
4,AGO,SNM.ind.1995,24.672298,1995
...,...,...,...,...
6155,WLF,SNM.ind.2022,9.452377,2022
6156,ESH,SNM.ind.2022,14.788210,2022
6157,YEM,SNM.ind.2022,34.527760,2022
6158,ZMB,SNM.ind.2022,36.637438,2022


**Note** This might seem like a slightly over-engineered solution.  We could've just used `s[-4:]` to get the last four characters of the string, which would've worked just as well in this case. But would it have? It's what I would call a fragile solution. It depends on the assumption that the data are all formatted the exact same way and don't contain any errors.  If there was a typo in one of the column names, it would break.  The regex solution is more robust, and it's generally good practice to right code that's as robust as you can make it within reason.  It didn't cost me much extra effort to write out the regex, and what it gains me is confidence that all of the year information was correctly extracted. This becomes especially important as the size of your dataset grows. For us, we only have a handful of years and only 6,000 or so observations, so we could run some summary statistics to make sure it looks right. But what if you had 6,000,000 observations? 6,000,000,000? Those are not unrealistic numbers to be working with these days, and running sanity checks at that scale becomes harder and harder.  Writing robust code can give us some confidence that things are working as expected.

## 5. Dropping Columns, Again

So, remember how I said that the `pandas` syntax was not intuitive above? This is the version that feels intuitive to me. We call the `drop` method again, this time with the `inplace` argument set to `True`. Instead of returning a new dataframe which we would then use to overwrite the old one, we just modify the existing dataframe, well, in place. 

In [130]:
SNM_long_indicators.drop(columns=['YearString'], inplace=True)
SNM_long_indicators

,CountryCode,Raw,Year
0,AFG,44.015857,1995
1,ALB,32.740191,1995
2,DZA,39.122933,1995
3,AND,-8888.000000,1995
4,AGO,24.672298,1995
...,...,...,...
6155,WLF,9.452377,2022
6156,ESH,14.788210,2022
6157,YEM,34.527760,2022
6158,ZMB,36.637438,2022


Similarly, we drop all of the observations which are negative, since those correspond to missing data in this dataset. Here, we're using a "boolean mask" or "logical indexing" to drop values. Read the expressions in the square brackets from the inside out:
1. `SNM_raw_indicators["Raw"] < 0` returns a boolean series which is `True` for all values in `SNM_raw_indicators["Raw"]` which are less than 0 and `False` otherwise.
2. `SNM_raw_indicators[SNM_raw_indicators["Raw"] < 0]` returns a dataframe which contains only the rows of `SNM_raw_indicators` for which the boolean series is `True`. We extract the index (row number) with the `.index` attribute.
3. `SNM_raw_indicators.drop(SNM_raw_indicators[SNM_raw_indicators["Raw"] < 0].index)` returns a dataframe which is the same as `SNM_raw_indicators` but without the rows we dropped.

In [131]:
SNM_long_indicators.drop(SNM_long_indicators[SNM_long_indicators['Raw'] < 0].index, inplace=True)
SNM_long_indicators

,CountryCode,Raw,Year
0,AFG,44.015857,1995
1,ALB,32.740191,1995
2,DZA,39.122933,1995
4,AGO,24.672298,1995
6,ATG,2.418811,1995
...,...,...,...
6155,WLF,9.452377,2022
6156,ESH,14.788210,2022
6157,YEM,34.527760,2022
6158,ZMB,36.637438,2022


## 6. Assigning a Variable

Finally, we create the `IndicatorCode` column, which contains our six letter indicator code, which is the last identifying piece of information we need to identify our varaibles.  This data is now ready to upload into the SSPI database!

In [132]:
SNM_long_indicators['IndicatorCode'] = 'NITROG'

## 7. Loading the Data

Finally, we can load the data into the SSPI Database. If you `flask run` your local app, you'll be able to watch as the data appear in your database.  In order to load into the live website, you'll have to manually call `database.authenticate()` and then `database.load_production()`.

In [133]:
database.load_data(SNM_long_indicators, "NITROG")

[{"CountryCode":"AFG","Raw":44.0158569791,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"ALB","Raw":32.7401912947,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"DZA","Raw":39.1229334047,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"AGO","Raw":24.6722980603,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"ATG","Raw":2.4188113759,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"ARG","Raw":75.1039100868,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"ARM","Raw":46.1770025135,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"AUS","Raw":55.2817248497,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"AUT","Raw":61.60946536,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"AZE","Raw":39.6523754854,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"BHS","Raw":13.1628842501,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"BHR","Raw":36.1543216542,"Year":"1995","IndicatorCode":"NITROG"},{"CountryCode":"BGD","Raw":31.

<Response [200]>

# Conclusion

In just a few (hopefully relatively straightforward) steps, we've gone from completely raw data to something that can be directly put into the SSPI database. We have preserved the original raw data in its native format, and every single step we've taken to go from raw data to final indicator data is documented in one place. When it comes time to replicate the indicator and you've totally forgotten what you've done, all of that information is right here in the notebook. Replication is as quick as downloading the right dataset and clicking the "Run All" button at the bottom.